In [1]:
import math
import operator
import warnings
import scipy.spatial
import tkinter
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy
import seaborn as sns
import sklearn
from pathlib import Path
from scipy import stats
from scipy.special import boxcox1p
from scipy.stats import kendalltau
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler, RobustScaler, QuantileTransformer, Normalizer

warnings.filterwarnings("ignore", category=UserWarning, module="matplotlib")

sns.set_context('notebook')
sns.set_style('white')
sns.set(color_codes='Blue', style="whitegrid")
sns.set_style("whitegrid", {'axes.grid': False})
sns.set_context(rc={'patch.linewidth': 0.0})
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
# Objective:
# Create a merged dataset of testing and mapping data and then determine which features are correlated to the target variable 'WPE'.

In [ ]:
# Main functions

def corr_results(df):
    """
    Iterate over all columns in dataframe and find column pairs that have significant correlations.
    Ignore column pairs that have the same str in name (i.e. "Power_1" & "Power_2")
    :param df: Dataframe object
    :return: Pairs of columns that have significant correlations.
    """
    corr_dict = {}
    for c1 in df.columns[0:-1]:
        for c2 in df.loc[:, c1:].columns:
            c1_str = (c1.split('_'))[0]
            c2_str = (c2.split('_'))[0]
            if c2 != c1:
                if c1_str != c2_str:
                    if c1.split('(')[0] == "Voltage" and c2_str in ("Power", "WPE", "WP") or \
                        c1.split('(')[0] == "Voltage" and c2.split('(')[0] == "Current" or \
                            c1.split('(')[0] == "Current" and c2_str in ("Power", "WPE", "WP", "OP", "Emitt", "Emitt:OP") or \
                            (c1.split('(')[0] == "Current" and c2.split('(')[0] == "Current") or \
                            (c1.split('(')[0] == "Current" and c2_str in ("OP", "Emitt", "Emitt:OP")) or \
                            (c1_str == "Power" and c2_str in ("WPE", "WP")) or \
                            (c1_str == "WPE" and c2_str == "WP") or \
                            (c1_str == "Emitt:OP" and c2_str in ("OP", "Emitt")) or (c1_str == "Emitt:OP" and c2.split('(')[0] == "Current"):
                        # print(c1, c2)
                        pass
                    else:
                        corr = df.loc[:, c1].corr(df.loc[:, c2], method='spearman')
                        if ((corr >= .5) | (corr <= -.5)) & (corr != 1.000):
                            corr_dict[c1 + ':' + c2] = corr
                            # print(c1_str, c2_str)
                    # print('Correlation:', c1, c2, '=', corr)
    results = pd.DataFrame(corr_dict.items(), columns=['Columns', 'Correlation']).sort_values(by='Correlation',ignore_index=True)
    return results

def corr_target(df, col_name):
    """
    Calculates correlation between the desired feature and all df columns
    """
    feature_target_corr = {}
    for col in df:
        if col_name != col:
            corr, p_val = kendalltau(df[col], df[col_name])
            if ((corr >= .2) | (corr <= -.2)) & (corr != 1.000):
                feature_target_corr[col + ':' + col_name] = corr
    results = pd.DataFrame(feature_target_corr.items(), columns=['Columns', 'Correlation']).sort_values(
        by='Correlation', ignore_index=True)
    return results


def get_duplicate_columns(df):
    """
    Iterate over all columns in dataframe and find the columns that have duplicate contents.
    :param df: Dataframe object
    :return: List of columns whose contents are duplicates.
    """
    duplicate_col_names = set()
    # Iterate over all the columns in dataframe
    for x in range(df.shape[1]):
        # Select column at xth index.
        col = df.iloc[:, x]
        # Iterate over all the columns in DataFrame from (x+1)th index till end
        for y in range(x + 1, df.shape[1]):
            # Select column at yth index.
            otherCol = df.iloc[:, y]
            # Check if two columns at x & y index are equal
            if col.equals(otherCol):
                duplicate_col_names.add(df.columns.values[y])
    return list(duplicate_col_names)


def single_feature_plot(df, feature):
    """
    :param df: Dataframe object, feature: numerical column
    :return: Boxplot and distplot of single numerical feature
    """
    sns.set(color_codes='Blue', style="whitegrid")
    sns.set_style("whitegrid", {'axes.grid': False})
    sns.set_context(rc={'patch.linewidth': 0.0})
    f, (ax1, ax2) = plt.subplots(2, gridspec_kw={"height_ratios": (.15, .65)})
    filtered = df.loc[~np.isnan(df[feature]), feature]
    sns.boxplot(filtered, color='steelblue', ax=ax1)
    sns.distplot(filtered, kde=True, hist=True, kde_kws={'linewidth': 1}, color='steelblue', ax=ax2)
    plt.show()


def multiple_features_plots(df, plot_type, numerical_cols=None):
    """
    :param: df: Dataframe object
    :param: plot_type: boxplot or histogram
    :param: numerical_cols: list of features to plot. if None, all numerical features in data frame will be plotted.
    :return: Boxplot or histogram of all numerical features in df (unless numerical_cols is specified)
    """
    if numerical_cols is None:
        cols = df.select_dtypes(include=np.number).columns.tolist()
    else:
        cols = numerical_cols
    num_plots = len(cols)
    num_cols = math.ceil(np.sqrt(num_plots))
    num_rows = math.ceil(num_plots / num_cols)
    fig = plt.figure(figsize=(24, 15))
    for i in range(len(cols)):
        var = cols[i]
        sub = fig.add_subplot(num_rows, num_cols, i + 1)
        sub.set_xlabel(var)
        if plot_type == "boxplot":
            sns.boxplot(df[var], color='steelblue')
        elif plot_type == "histogram":
            sns.distplot(df[var], kde=True, hist=True)
        else:
            print("Plot type is incorrect.")
    plt.tight_layout()


def skew_values(df):
    """
    Calculates skewness of all dataframe columns
    """
    return df.skew(axis=0, skipna=True)

def color_row(row):
    """
    Color code skewness table, so it's easier to identify abnormal distributions
    """
    return ['background-color: #8a2be2' if (v < 0.5) & (v > -0.5)
            else 'background-color: #a2a2d0' if (v > -1) & (v < -0.5) | (v < 1) & (v > 0.5) else "" for i, v in
            row.items()]


def transformations(df):
    """
    Outputs transformed dataframes.
    :param: df
    :return: df, Log df, Box-Cox df, Sqrt df, QuantileTransformer (Uniform) df, QuantileTransformer (Normal) df, Normalizer() df
    """
    numeric_col = df.select_dtypes(include=np.number).columns.tolist()
    df2 = df[numeric_col]

    if (df2 <= 0).values.any():
        log_df = df2.apply(lambda x: np.log(x + 0.001))
        box_cox_df = df2.apply(lambda x: boxcox1p(x, 0.25))
    else:
        print("everything is positive")
        log_df = df2.apply(lambda x: np.log(x))
        box_cox_df = df2.apply(lambda x: stats.boxcox(x)[0])

    sqrt_df = df2.apply(lambda x: np.sqrt(x))
    quant_uniform_df = pd.DataFrame(QuantileTransformer(output_distribution="uniform").fit_transform(df2),
                                    columns=df2.columns)
    quant_normal_df = pd.DataFrame(QuantileTransformer(output_distribution="normal").fit_transform(df2),
                                   columns=df2.columns)
    normalizer_df = pd.DataFrame(Normalizer().fit_transform(df2), columns=df2.columns)

    return df2, log_df, box_cox_df, sqrt_df, quant_uniform_df, quant_normal_df, normalizer_df


def compare_skewness(df):
    """
    :param: df
    :return: table containing the skewness of each transformation distribution for all df columns
    cells highlighted in purple have approximately symmetric distributions
    cells highlighted in lilac have moderately skewed distributions
    """
    df, log_df, box_cox_df, sqrt_df, quant_uniform_df, quant_normal_df, normalizer_df = transformations(df)
    sk_table = pd.concat(
        [skew_values(df), skew_values(log_df), skew_values(box_cox_df), skew_values(sqrt_df),
         skew_values(quant_uniform_df), skew_values(quant_normal_df),
         skew_values(normalizer_df)], axis=1)
    sk_table = sk_table.rename(
        columns={0: 'Regular', 1: "Log", 2: "Box-cox", 3: "Sqrt", 4: "QuantileTransformer (Uniform)",
                 5: "QuantileTransformer (Normal)", 6: "Normalizer"})

    print("If skewness is less than -1 or greater than 1, the distribution is highly skewed."
          "\nIf skewness is between -1 and -0.5 or between 0.5 and 1, the distribution is moderately skewed."
          "\nIf skewness is between -0.5 and 0.5, the distribution is approximately symmetric.")
    sk_table = sk_table.style.apply(color_row,
                                    subset=['Regular', "Log", "Box-cox", "Sqrt", "QuantileTransformer (Uniform)",
                                            "QuantileTransformer (Normal)", "Normalizer"], axis=1)
    return sk_table


def plot_transformations(df, transformation_list, numerical_cols=None):
    """
    :param: df
    :param: transformation_list options: ['regular', 'log', 'boxcox', 'sqrt', 'QuantileTransformer (Uniform)', 'QuantileTransformer (Normal)', 'Normalizer']
    :param: numerical_cols: list of features to plot. if None, all numerical features in data frame will be plotted.
    :return: Histograms representing specified transformations of all numerical features in df (unless numerical_cols is specified)
    """
    fig = plt.figure(figsize=(24, 15))

    if numerical_cols is None:
        df, log_df, box_cox_df, sqrt_df, quant_uniform_df, quant_normal_df, normalizer_df = transformations(df)
        num_plots = len(df.axes[1])
    else:
        df2 = df[numerical_cols]
        df, log_df, box_cox_df, sqrt_df, quant_uniform_df, quant_normal_df, normalizer_df = transformations(df2)
        num_plots = len(df.axes[1])

    d = {'regular': df, 'log': log_df, 'boxcox': box_cox_df, 'sqrt': sqrt_df,
         "QuantileTransformer (Uniform)": quant_uniform_df, "QuantileTransformer (Normal)": quant_normal_df,
         "Normalizer": normalizer_df}

    df_list = [d[x] for x in transformation_list]

    num_cols = math.ceil(np.sqrt(num_plots))
    num_rows = math.ceil(num_plots / num_cols)
    for i in range(num_plots):
        var = df.columns.tolist()[i]
        sub = fig.add_subplot(num_rows, num_cols, i + 1)
        sub.set_xlabel(var)
        for df in df_list:
            sns.distplot(df[var], kde=True, hist=True)
    fig.legend(labels=transformation_list)
    plt.tight_layout()


def plot_scalars(df, numerical_cols=None):
    """
    :param: df
    :param: numerical_cols: list of features to plot. if None, all numerical features in data frame will be plotted.
    :return: Histograms representing 4 different scalars of all numerical features in df (unless numerical_cols is specified)
    """
    fig = plt.figure(figsize=(24, 15))

    if numerical_cols is None:
        numeric_cols = df.select_dtypes(include=np.number).columns.tolist()
        df2 = df[numeric_cols]
    else:
        df2 = df[numerical_cols]

    standard_df = pd.DataFrame(StandardScaler().fit_transform(df2.values), columns=df2.columns, index=df2.index)
    min_max_df = pd.DataFrame(MinMaxScaler().fit_transform(df2.values), columns=df2.columns, index=df2.index)
    robust_df = pd.DataFrame(RobustScaler().fit_transform(df2.values), columns=df2.columns, index=df2.index)

    num_plots = len(df2.axes[1])
    num_cols = math.ceil(np.sqrt(num_plots))
    num_rows = math.ceil(num_plots / num_cols)
    for i in range(num_plots):
        var = df2.columns.tolist()[i]
        sub = fig.add_subplot(num_rows, num_cols, i + 1)
        sub.set_xlabel(var)
        sns.distplot(df2[var], kde=True, hist=True)
        sns.distplot(standard_df[var], kde=True, hist=True)
        sns.distplot(min_max_df[var], kde=True, hist=True)
        sns.distplot(robust_df[var], kde=True, hist=True)

    fig.legend(labels=['Regular', 'Standard', 'Min Max', 'Robust'])
    plt.tight_layout()


def one_hot_encoder(data, feature):
    """
    :param: df, numerical column
    :return: encoded column
    """
    oh = OneHotEncoder()
    oh_df = pd.DataFrame(oh.fit_transform(data[[feature]]).toarray())
    oh_df.columns = oh.get_feature_names()
    for col in oh_df.columns:
        oh_df.rename({col: f'{feature}_' + col.split('_')[1]}, axis=1, inplace=True)
    return oh_df


def outliers_iqr(df, numerical_cols=None):
    """
    Uses IQR to identify outlier data points for each column.
    Prints entire index list of outliers and the common index list of outliers for all features.
    :param df: Data frame to analyze for outliers.
    :param numerical_cols: If argument is None, all numerical columns will be analyzed for outliers.
    :return: outliers_df which contains just the outliers, cleaned_df which is resulting data frame with no outliers.
    """
    outliers_list = []
    if numerical_cols is None:
        col_list = df.select_dtypes(include=np.number).columns.tolist()
    else:
        col_list = numerical_cols
    for feature in col_list:
        Q1 = np.percentile(df[feature], 25)
        Q3 = np.percentile(df[feature], 75)
        step = (Q3 - Q1) * 1.5
        print("Data points considered outliers for the feature '{}':".format(feature))
        outliers = list(df[~((df[feature] >= Q1 - step) & (df[feature] <= Q3 + step))].index.values)
        print((df[feature].loc[df[feature].index[outliers]]).tolist())
        outliers_list.extend(outliers)

    print("\n Index List of Outliers -> {}".format(outliers_list))
    duplicate_outliers_list = list(set([x for x in outliers_list if outliers_list.count(x) >= 2]))
    duplicate_outliers_list.sort()
    print("\n Index List of Common Outliers -> {}".format(duplicate_outliers_list))

    # Select the indices for data points you wish to remove
    outliers_df = df.iloc[duplicate_outliers_list, :]
    cleaned_df = df.drop(df.index[duplicate_outliers_list]).reset_index(drop=True)
    return outliers_df, cleaned_df

def calc_drop(res):
    """
    Calculates instances of multicollineatiry between pairs of columns in data frame.
    """
    # All variables with correlation > cutoff
    all_corr_vars = list(set(res['v1'].tolist() + res['v2'].tolist()))

    # All unique variables in drop column
    poss_drop = list(set(res['drop'].tolist()))

    # Keep any variable not in drop column
    keep = list(set(all_corr_vars).difference(set(poss_drop)))

    # Drop any variables in same row as a keep variable
    p = res[ res['v1'].isin(keep)  | res['v2'].isin(keep) ][['v1', 'v2']]
    q = list(set(p['v1'].tolist() + p['v2'].tolist()))
    drop = (list(set(q).difference(set(keep))))

    # Remove drop variables from possible drop
    poss_drop = list(set(poss_drop).difference(set(drop)))

    # subset res dataframe to include possible drop pairs
    m = res[ res['v1'].isin(poss_drop)  | res['v2'].isin(poss_drop) ][['v1', 'v2','drop']]

    # remove rows that are decided (drop), take set and add to drops
    more_drop = set(list(m[~m['v1'].isin(drop) & ~m['v2'].isin(drop)]['drop']))
    for item in more_drop:
        drop.append(item)

    return drop


def corr_x_new(df, cut = 0.9) :
    """
    Decides which columns to drop from data frame due to multicollineaity
    """
    # Get correlation matrix and upper triangle
    corr_mtx = df.corr().abs()
    avg_corr = corr_mtx.mean(axis = 1)
    up = corr_mtx.where(np.triu(np.ones(corr_mtx.shape), k=1).astype(np.bool))
    dropcols = list()
    res = pd.DataFrame(columns=(['v1', 'v2', 'v1.target',
                                 'v2.target','corr', 'drop' ]))
    for row in range(len(up)-1):
        col_idx = row + 1
        for col in range (col_idx, len(up)):
            if corr_mtx.iloc[row, col] > cut:
                if avg_corr.iloc[row] > avg_corr.iloc[col]:
                    dropcols.append(row)
                    drop = corr_mtx.columns[row]
                else:
                    dropcols.append(col)
                    drop = corr_mtx.columns[col]
                s = pd.Series([ corr_mtx.index[row], up.columns[col], avg_corr[row], avg_corr[col],
                                up.iloc[row,col], drop], index = res.columns)
                res = res.append(s, ignore_index = True)
    print(res)
    dropcols_names = calc_drop(res)
    return dropcols_names


In [ ]:
# Functions for creating merged dataset

def rotate(vector, theta, rotation_around=None) -> np.ndarray:
    """
    reference: https://en.wikipedia.org/wiki/Rotation_matrix#In_two_dimensions
    :param vector: list of length 2 OR
                   list of list where inner list has size 2 OR
                   1D numpy array of length 2 OR
                   2D numpy array of size (number of points, 2)
    :param theta: rotation angle in degree (+ve value of anti-clockwise rotation)
    :param rotation_around: "vector" will be rotated around this point,
                    otherwise [0, 0] will be considered as rotation axis
    :return: rotated "vector" about "theta" degree around rotation
             axis "rotation_around" numpy array
    """
    vector = np.array(vector)
    if vector.ndim == 1:
        vector = vector[np.newaxis, :]

    if rotation_around is not None:
        vector = vector - rotation_around

    vector = vector.T
    theta = np.radians(theta)

    rotation_matrix = np.array([
        [np.cos(theta), np.sin(theta)],
        [-np.sin(theta), np.cos(theta)]
    ])

    output: np.ndarray = (rotation_matrix @ vector).T

    if rotation_around is not None:
        output = output + rotation_around
    return output.squeeze()


def get_tracking_data():
    """
    Ask user to select tracking .dat file
    Delete "bad" data points from this file that are out of spec
    Rotate (x,y) 45 degrees clockwise
    Round rotated data points to two decimal points
    """
    root = tkinter.Tk()
    root.withdraw()
    while True:
        try:
            print("Select a tracking file...")
            tracking_dat = tkinter.filedialog.askopenfilename(title="Select a tracking .dat file")
            tracking_dat_filename = Path(tracking_dat).stem
            tracking_df = pd.read_csv(tracking_dat, delimiter=',', skiprows=28, header=None)
            # Raise exception if wrong type of file was selected
            if len(tracking_df.columns) != 6:
                raise Exception
            tracking_df_filter = tracking_df[tracking_df["I_S"] > 100]
            tracking_df_filter = tracking_df_filter.reset_index(drop=True)
            tracking_x_y = tracking_df_filter[["X(mm)", "Y(mm)"]].to_numpy()
            tracking_x_y_rotate = rotate(tracking_x_y, -45)
            tracking_df_filter["Tracking X(mm) Rotate"], tracking_df_filter["Tracking Y(mm) Rotate"] = tracking_x_y_rotate[:, 0], tracking_x_y_rotate[:, 1]
            tracking_df_filter["Tracking X(mm) Rotate"] = tracking_df_filter["Tracking X(mm) Rotate"].apply(lambda x: round(x, 2))
            tracking_df_filter["Tracking Y(mm) Rotate"] = tracking_df_filter["Tracking Y(mm) Rotate"].apply(lambda x: round(x, 2))
            return tracking_dat_filename, tracking_df_filter
        except:
            print("You must select a valid tracking file.")


def get_results_data():
    """
    Ask user to select results .dat file
    Delete "bad" data points from this file that are out of spec
    Rotate (x,y) 45 degrees clockwise
    Round rotated data points to two decimal points
    """
    root = tkinter.Tk()
    root.withdraw()
    while True:
        try:
            print("Select a results .dat file...")
            results_dat = tkinter.filedialog.askopenfilename(title="Select a results .dat file")
            results_dat_filename = Path(results_dat).stem
            results_df = pd.read_csv(results_dat, delimiter=',', skiprows=30, header=None)
            # Raise exception if wrong type of file was selected
            if len(results_df.columns) != 8:
                raise Exception
            results_df_filter = results_df[(results_df['Dip'] < 1495) & (results_df['Dip'] > 1220)]
            results_df_filter = results_df_filter.reset_index(drop=True)
            results_x_y = results_df_filter[["X(mm)", "Y(mm)"]].to_numpy()
            results_x_y_rotate = rotate(results_x_y, -45)
            results_df_filter["Results X(mm) Rotate"], results_df_filter["Results Y(mm) Rotate"] = results_x_y_rotate[:,
                                                                                     0], results_x_y_rotate[:, 1]
            results_df_filter["Results X(mm) Rotate"] = results_df_filter["Results X(mm) Rotate"].apply(lambda x: round(x, 2))
            results_df_filter["Results Y(mm) Rotate"] = results_df_filter["Results Y(mm) Rotate"].apply(lambda x: round(x, 2))
            return results_dat_filename, results_df_filter
        except:
            print("You must select a valid results dat file.")


def get_map():
    """
    Upload map into dataframe
    """
    map_df = pd.read_excel(r"..\map.xlsx")
    map_df['Map x (mm)'] = map_df['Map x (mm)'].apply(lambda x: round(x, 2))
    map_df['Map y (mm)'] = map_df['Map y (mm)'].apply(lambda x: round(x, 2))
    return map_df


def nearest_neighbors(coordinates, map_df):
    """
    Determine the nearest set of rotated Tracking/Results coordinates for each set of (x,y) coordinates on the  map
    """
    map_df = map_df[["Map x (mm)", "Map y (mm)"]].to_numpy()
    # Subset last two columns of coordinates which are the rotated X & Y coordinates
    coordinates_sub = coordinates.iloc[:, -2:]
    # Convert rotated X & Y coordinates columns to numpy array
    coordinates_sub = coordinates_sub.to_numpy()
    list_of_dicts = []
    for row in map_df:
        # construct a kd-tree of rotated coordinates
        map_tree = scipy.spatial.cKDTree(coordinates_sub)
        # find nearest neighbors for each map coordinate within kd-tree of tracking/results rotated coordinates
        distance, index = map_tree.query(row)
        # Create a row entry in data frame
        cols = ["Map X", "Map Y", "X Rotate", "Y Rotate", "Distance"]
        map_x = row[0]
        map_y = row[1]
        coords_x = (coordinates_sub[index]).flat[0]
        coords_y = (coordinates_sub[index]).flat[1]
        results = [map_x, map_y, coords_x, coords_y, distance]
        results_dict = dict(zip(cols, results))
        list_of_dicts.append(results_dict)
    results_df = pd.DataFrame(list_of_dicts)
    # print(results_df)
    return results_df


In [ ]:
# Creating mapped dataset

tracking_dat_filename, tracking_df = get_tracking_data()
results_dat_filename, results_df = get_results_data()
map_df = get_map()

tracking_map_df = nearest_neighbors(tracking_df, map_df)
results_map_df = nearest_neighbors(results_df, map_df)

# Create resulting dataframes

# Tracking
tracking_merge_df = pd.merge(map_df, tracking_map_df, on=['Map x (mm)', 'Map y (mm)'], how="left")
final_tracking_merge_df = pd.merge(tracking_merge_df, tracking_df, on=['Tracking X(mm) Rotate', 'Tracking Y(mm) Rotate'], how="left")

# Results
results_merge_df = pd.merge(map_df, results_map_df, on=['Map x (mm)', 'Map y (mm)'], how="left")
final_results_merge_df = pd.merge(results_merge_df, results_df, on=['Results X(mm) Rotate', 'Results Y(mm) Rotate'],how="left")


# Combine Tracking & Results to create mapped_df
mapped_df = pd.merge(final_tracking_merge_df, final_results_merge_df,
                      on=['Gx', 'Gy', 'Device', 'Map x (mm)', ' Map y (mm)'], how="left")


In [ ]:
# Test data
test_df = pd.read_excel(r"..\test_data.xlsx")
print("Test df shape:", test_df.shape)

# Merge test data with mapped data to create df for project
merged_df = test_df.merge(mapped_df, left_on=['X', 'Y', 'ID'], right_on=['Gx', 'Gy', 'ID'], how='left')

print("Merged df shape:", merged_df.shape)
print(merged_df['ID'].value_counts())

In [ ]:
# Find duplicate columns in merged df
duplicateColumnNames = get_duplicate_columns(merged_df)
print('Duplicate Columns:')
for col in duplicateColumnNames:
    print(col)
corr_df = merged_df.drop(columns=duplicateColumnNames)

In [ ]:
# Print columns that only have one constant value
# If a column only has one unique value, then it is not important to keep in data frame for modeling purposes
bad_cols = (corr_df.columns[corr_df.nunique() <= 2])

# Print columns to list to locate other irrelevant columns
print(corr_df.columns.tolist())

In [ ]:
# Drop columns that carry the same information & non-relevant columns
corr_df_drop = corr_df.copy()
corr_df_drop.drop(bad_cols, axis=1, inplace=True)

In [ ]:
# Replace negative values (data errors) and zero values with 0.00001 to make transformations easier

positive_df = corr_df_drop.copy()
num = positive_df._get_numeric_data()
num[num <= 0] = 0.00001
print(positive_df.min(axis=0))

In [ ]:
corr_df_filter = positive_df.copy()

# Identify data types
print(corr_df_filter.info())

# Check number of unique values in the object columns
print(corr_df_filter.nunique())

# Convert to categorical data types because these are fixed input variables
corr_df_filter['Temp'] = corr_df_filter['Temp'].astype('category')
corr_df_filter['Lamd'] = corr_df_filter['Lamd'].astype('category')
corr_df_filter['Dip'] = corr_df_filter['Dip'].astype('category')
corr_df_filter['ID'] = corr_df_filter['ID'].astype('category')

In [ ]:
# Overall Emitt/OP Plots

corr_df_filter_sort = corr_df_filter.sort_values(by=['Emitt'], ascending=True)

fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(20, 20))
emitter_values = corr_df_filter_sort['Emitt'].value_counts().sort_index().values
sns.countplot(x='Emitt', order=['6', '12', '25', '28', '34', '41', '48', '62'],
              data=corr_df_filter_sort, ax=ax1)
for i, p in enumerate(ax1.patches):
    height = p.get_height()
    ax1.text(p.get_x() + p.get_width() / 2., height + 0.1, emitter_values[i], ha="center")

oa_values = corr_df_filter_sort['OP'].value_counts().sort_index().values
sns.countplot(x='OP', order=['03k', '05k', '08k', '09k', '11k', '13k', '15k', '18k'],
              data=corr_df_filter_sort, ax=ax2)
for i, p in enumerate(ax2.patches):
    height = p.get_height()
    ax2.text(p.get_x() + p.get_width() / 2., height + 0.1, oa_values[i], ha="center")

sns.countplot(x='Emitt', color="gainsboro", order=['6', '12', '25', '28', '34', '41', '48', '62'],
              data=corr_df_filter_sort, ax=ax3)
for i, p in enumerate(ax3.patches):
    height = p.get_height()
    ax3.text(p.get_x() + p.get_width() / 2., height + 0.1, emitter_values[i], ha="center")
sns.countplot(x='Emitt', hue='OP',
              hue_order=['03k', '05k', '08k', '09k', '11k', '13k', '15k', '18k'],
              data=corr_df_filter_sort, ax=ax3)
plt.show()

In [ ]:
# Emitt/OP Plots within Recipe

fig, (ax4, ax5) = plt.subplots(2, 1, figsize=(20, 15))
recipe_values = corr_df_filter_sort['Recipes'].value_counts().sort_index().values
sns.countplot(x='Recipes', color="gainsboro",
              data=corr_df_filter_sort, ax=ax4)
for i, p in enumerate(ax4.patches):
    height = p.get_height()
    ax4.text(p.get_x() + p.get_width() / 2., height + 0.1, recipe_values[i], ha="center")
sns.countplot(x='Recipes', hue='OP',
              hue_order=['04um', '03k', '05k', '08k', '09k', '11k', '13k', '15k', '18k'],
              data=corr_df_filter_sort, ax=ax4)

sns.countplot(x='Recipes', color="gainsboro",
              data=corr_df_filter_sort, ax=ax5)
for i, p in enumerate(ax5.patches):
    height = p.get_height()
    ax5.text(p.get_x() + p.get_width() / 2., height + 0.1, recipe_values[i], ha="center")
sns.countplot(x='Recipes', hue='Emitt',
              hue_order=['6', '12', '19', '28', '34', '41', '48', '62'], data=corr_df_filter_sort, ax=ax5)

plt.show()

In [ ]:
# Grouping Recipe Name by Temperature

print(corr_df_filter.groupby(['Recipe Name'])['Temp'].value_counts().sort_index())

In [ ]:
# Fixed input variables: Emitt, OP, Current(mA), Dip, Lamd, Temperature, Recipes, Recipe Name, ID
# Converting Emitt, OP, Current(mA), Recipe, Recipe Name into categorical variables
# Dip, Lamd, Temperature were converted into categorical variables earlier

# Combo of (Emitt, OP)
# There are 41 unique combinations of Emitt and OP

corr_df_filter['Emitt:OP'] = corr_df_filter[['Emitt', 'OP']].apply(
    lambda row: ':'.join(row.values.astype(str)), axis=1)

corr_df_filter['Emitt:OP'] = corr_df_filter['Emitt:OP'].astype('category')

corr_df_filter = corr_df_filter.drop(['Emitt','OP'], axis=1)

# corr_df_filter['Emitt'] = corr_df_filter['Emitt'].str[:-1]
# corr_df_filter['OP'] = corr_df_filter['OP'].str[:-2]

corr_df_filter['Recipe Name'] = corr_df_filter['Recipe Name'].astype('category')
corr_df_filter['Recipes'] = corr_df_filter['Recipes'].astype('category')
corr_df_filter['ID'] = corr_df_filter['ID'].astype('category')
corr_df_filter['Emitt'] = corr_df_filter['Emitt'].astype('category')
corr_df_filter['OP'] = corr_df_filter['OP'].astype('category')

In [ ]:
# Current(mA)
# There are 37 unique combinations of fixed inputs chosen from Steps1-7
# Assign each unique combination to a letter
# Convert individual Current_Step columns into category variables to track individual steps

gp_current = (corr_df_filter.groupby(
    ['Current(mA)_1', 'Current(mA)_2', 'Current(mA)_Step3', 'Current(mA)_4',
     'Current(mA)_Step5', 'Current(mA)_6', 'Current(mA)_Step7']).size().reset_index(name='Freq'))
gp_current.drop(gp_current.columns[len(gp_current.columns) - 1], axis=1, inplace=True)
gp_current_keys = gp_current.values.tolist()


count = 0
d = {}
for i in gp_current_keys:
    if count == 26:
        count += 6
    if tuple(i) not in d:
        d[tuple(i)] = chr(ord('A') + count)
    count += 1


df = corr_df_filter.copy()
df['Current(mA)_StepType'] = df.apply(
    lambda row: d[(row['Current(mA)_1'], row['Current(mA)_2'], row['Current(mA)_3'],
                   row['Current(mA)_4'], row['Current(mA)_5'], row['Current(mA)_6'],
                   row['Current(mA)_7'])], axis=1)

for col in ['Current(mA)_StepType', 'Current(mA)_1', 'Current(mA)_2',
            'Current(mA)_3', 'Current(mA)_4',
            'Current(mA)_5', 'Current(mA)_6', 'Current(mA)_7']:
    df[col] = df[col].astype('category')

In [24]:
# Categorical plots

In [ ]:
# ID plot

wafer_sort = df.sort_values(by=['ID'], ascending=True)

fig, (ax1) = plt.subplots(1, 1, figsize=(30, 15))
wafer_values = wafer_sort['ID'].value_counts().sort_index().values
sns.countplot(x='ID', data=wafer_sort, ax=ax1)
for i, p in enumerate(ax1.patches):
    height = p.get_height()
    ax1.text(p.get_x() + p.get_width() / 2., height + 0.1, wafer_values[i], ha="center")
plt.show()

In [26]:
# Plot Emitt:OP Counts

fig, (ax) = plt.subplots(1, 1, figsize=(30, 15))
emoa_sort = df.sort_values(by=['Emitt:OP'], ascending=True)
emoa_sort_val = emoa_sort['Emitt:OP'].unique()
print(emoa_sort_val)

sns.countplot(x='Emitt:OP', data=emoa_sort, order=emoa_sort_val, ax=ax)
ax.set_xticklabels(ax.get_xticklabels(), rotation=40, ha="right")
ax.set_title('Emitt:OP Count')
plt.tight_layout()
plt.show()

In [ ]:
# X & Y by Temperature

g2 = sns.FacetGrid(df, col="Temp", height=10, aspect=1)
g2.map(sns.scatterplot, "X", "Y")
plt.show()

In [ ]:
# Plot Current(mA)_StepType

sns.set(rc={'figure.figsize': (20, 10)})
g3 = sns.countplot(x='Current(mA)_StepType', data=df)
g3.set_title('Current(mA)_StepType Count')
plt.show()


In [ ]:
# Lamd/Dip

print("Most frequent Lamd values:", df['Lamd'].value_counts().index.tolist())
g4 = sns.countplot(x='Lamd', data=df)
g4.set_title('Lamd Input Counts')
plt.show()

print("Most frequent Dip values:", df['Dip'].value_counts().index.tolist())
g5 = sns.countplot(x='Dip', data=df)
g5.set_title('Dip Input Counts')
plt.show()

In [ ]:
# Temp plot

g6 = sns.countplot(x='Temp', data=df)
g6.set_title('Temp Counts')
plt.show()

In [ ]:
# Recipes plot

g7 = sns.countplot(x='Recipes', data=df)
g7.set_title('Recipes')
plt.show()

In [32]:
# Numerical plots


In [ ]:
# Y variation within Fp-Dip and Lamd_2

sns.lineplot(data=df, x="Y", y="Dip")
plt.show()
sns.lineplot(data=df, x="Y", y="Lamd_2")
plt.show()

In [ ]:
# Boxplots showing relationships between all categorical variables and WPE

categorical_features = (df.select_dtypes(include=['category']).columns.tolist())

fig = plt.figure(figsize=(20, 50))
for i in range(len(categorical_features)):
    var = categorical_features[i]
    fig.add_subplot(11, 4, i + 1)
    sns.boxplot(y=df['WPE'], x=df[var])
plt.title('Correlations Between WPE and all Categorical Variables')
plt.tight_layout()
plt.show()

In [ ]:
# Correlation matrix between all numerical variables

plt.figure(figsize=(30, 15))
mask = np.zeros_like(df.corr(), dtype=np.bool)
mask[np.triu_indices_from(mask)] = True
sns.heatmap(df.corr(), cmap=sns.diverging_palette(20, 220, n=200),
            mask=mask, annot=True, center=0)
plt.title('Correlations Between all Numerical Variables')
plt.show()

In [38]:
# Remove 'Recipe Name' and 'Recipes' column

df = df.drop(columns=['Recipe Name', 'Recipes'])

In [39]:
# One hot encode

emitt_op_encode = one_hot_encoder(df, 'Emitt:OP')
# fpdip_encode = one_hot_encoder(df, 'Dip')
# lambda_encode = one_hot_encoder(df, 'Lamd')
current_encode = one_hot_encoder(df, 'Current(mA)_StepType')
temp_encode = one_hot_encoder(df, 'Temp')
emitt_encode = one_hot_encoder(df, 'Emitt')
op_encode = one_hot_encoder(df, 'OP')

one_hot_encode = pd.concat([
emitt_op_encode, current_encode, temp_encode, emitt_encode, op_encode], axis=1)
one_hot_encode.reset_index(drop=True, inplace=True)
df.reset_index(drop=True, inplace=True)
encoded_df = pd.merge(df, one_hot_encode, left_index=True, right_index=True)
encoded_df = encoded_df.drop([
    'Emitt:OP', 'Current(mA)_StepType', 'Temp', 'Emitt', 'OP',], axis=1)
encoded_df.columns = encoded_df.columns.str.replace(']', ')')
print(encoded_df.shape)

In [ ]:
num_cols = ['X', 'Y', 'Voltage(V)_1', 'Voltage(V)_2', 'Voltage(V)_3', 'Voltage(V)_4',
 'Voltage(V)_5', 'Voltage(V)_6', 'Voltage(V)_7', 'Power_1', 'Power_2',
 'Power_3', 'Power_4', 'Power_5', 'Power_6', 'Power_7', 'WPE', 'Ith',
 'Lamd_2', 'SpecWR', 'SpecW', 'Ith_I', 'Dense_V',
 'Dense_VI', 'Peak_I', 'I_S']

# Correlation Matrix for numerical variables

plt.figure(figsize=(30, 15))
mask = np.zeros_like(df[num_cols].corr(), dtype=np.bool)
mask[np.triu_indices_from(mask)] = True
sns.heatmap(df[num_cols].corr(), cmap=sns.diverging_palette(20, 220, n=200),
            mask=mask, annot=True, center=0)
plt.show()

In [ ]:
# Remove outliers from numerical columns using IQR method

df2 = df.copy()
df2 = df2.reset_index(drop=True)
df2_outliers, df2_cleaned = outliers_iqr(df2, numerical_cols = num_cols)

# Boxplots before outlier removal
multiple_features_plots(df2, "boxplot", num_cols)

# Boxplots after outlier removal
multiple_features_plots(df2_cleaned, "boxplot", num_cols)

In [ ]:
# Initial correlation results
corr_results(df2_cleaned)


In [ ]:
# Calculate skewness of each variable for various distributions
compare_skewness(df2_cleaned[num_cols])


In [66]:
# Transform necessary variables using box-cox transformation
transform_cols = ['Power_1',  'Ith', 'SpecWR', 'Ith_I']
# Plot transformation
plot_transformations(df2_cleaned, numerical_cols=transform_cols, transformation_list=['regular', 'boxcox'])

df2_transformed  = df2_cleaned.copy()
df2_transformed[transform_cols] = df2_transformed[transform_cols].apply(lambda x: stats.boxcox(x)[0])

In [81]:
# Shuffle data frame
df2_shuffled = df2_transformed.sample(frac=1, random_state=42).reset_index(drop=True)


In [ ]:
# Correlation with target variable 'WPE'

df2_shuffled_corr = corr_target(df2_shuffled, 'WPE')
print(df2_shuffled_corr)


In [ ]:
# Imports for ML

import shap
from sklearn import metrics
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
from sklearn.ensemble import AdaBoostRegressor, RandomForestRegressor
from sklearn.feature_selection import SelectKBest, mutual_info_regression
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel, RBF, Matern, RationalQuadratic
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet, SGDRegressor
from sklearn.model_selection import *
from sklearn.pipeline import Pipeline
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor


In [ ]:
# Designate 'WPE' as the target variable and other variables as predictors

wpe_df_x = df2_shuffled.drop('WPE', axis=1)
wpe_df_y = df2_shuffled['WPE']

# Split into training and testing data sets
wpe_x_train, wpe_x_test, wpe_y_train, wpe_y_test = train_test_split(wpe_df_x, wpe_df_y, test_size=0.2, random_state=42)
print(f"Train set has {wpe_x_train.shape[0]} records out of {len(df2_shuffled)} which is {round(wpe_x_train.shape[0] / len(df2_shuffled) * 100)}%")
print(f"Test set has {wpe_x_test.shape[0]} records out of {len(df2_shuffled)} which is {round(wpe_x_test.shape[0] / len(df2_shuffled) * 100)}%")

print("Shape of X_train: ", wpe_x_train.shape)
print("Shape of X_test: ", wpe_x_test.shape)
print("Shape of y_train: ", wpe_y_train.shape)
print("Shape of y_test: ", wpe_y_test.shape)

In [ ]:
# Scale numerical features using StandardScaler approach

numerical_features = ['X', 'Y', 'Ith', 'Lamd_2', 'SpecWR', 'SpecW', 'Ith_I',
 'Dense_V', 'Dense_VI', 'Lamd', 'Peak_I', 'I_S', 'Dip', 'SB Delta']

scaler = StandardScaler()
wpe_x_train[numerical_features] = scaler.fit_transform(wpe_x_train[numerical_features])
wpe_x_test[numerical_features] = scaler.transform(wpe_x_test[numerical_features])


In [ ]:
# Drop variables from dataset with high multicollinearity

corr_drop_cols = corr_x_new(wpe_x_train, cut = 0.9)
print(corr_drop_cols)
wpe_x_train.drop(columns=corr_drop_cols, axis=1, inplace=True)


In [ ]:
# Mutual information is calculated between two variables and measures the reduction in uncertainty for one variable given a known value of the other variable

cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
model = LinearRegression()
fs = SelectKBest(score_func=mutual_info_regression)
pipeline = Pipeline(steps=[('sel',fs), ('lr', model)])

# define the grid
grid = dict()

# Initial test for range of optimal features
grid['sel__k'] = [i for i in range(wpe_x_train.shape[1]-60, wpe_x_train.shape[1]+1)]

# 113 total features
# 76 is the number of optimal features
# We will make the range (50,100) for future testing of optimal features
# 0.019 MAE begins at 56 features
# grid['sel__k'] = [i for i in range(wpe_x_train.shape[1]-63, wpe_x_train.shape[1]-12)]

# define the grid search
search = GridSearchCV(pipeline, grid, scoring='neg_mean_squared_error', n_jobs=-1, cv=cv)
results = search.fit(wpe_x_train, wpe_y_train)

print('Best MAE: %.3f' % results.best_score_)
print('Best Config: %s' % results.best_params_)
means = results.cv_results_['mean_test_score']
params = results.cv_results_['params']
for mean, param in zip(means, params):
    print(">%.3f with: %r" % (mean, param))

In [ ]:
# evaluation of a model using 76 optimal features

# feature selection
def select_features(X_train, y_train, X_test, k):
    fs = SelectKBest(score_func=mutual_info_regression, k=k)
    fs.fit(X_train, y_train)
    X_train_fs = fs.transform(X_train)
    X_test_fs = fs.transform(X_test)
    return X_train_fs, X_test_fs, fs

X_train_fs, X_test_fs, fs = select_features(wpe_x_train, wpe_y_train, wpe_x_test, k=76)
model = LinearRegression()
model.fit(X_train_fs, wpe_y_train)
yhat = model.predict(X_test_fs)
mae = mean_absolute_error(wpe_y_test, yhat)
print('MAE: %.3f' % mae)


In [ ]:
# Print 76 chosen features with their F-scores

names = wpe_x_train.columns.values[fs.get_support()]
scores = fs.scores_[fs.get_support()]
names_scores = list(zip(names, scores))
ns_df = pd.DataFrame(data = names_scores, columns=['Feat_names', 'F_Scores'])
ns_df_sorted = ns_df.sort_values(['F_Scores', 'Feat_names'], ascending = [False, True])
print(ns_df_sorted)


In [ ]:

# Calculate RMSE for every model type
# Pick 3 lowest RMSE and create those models

lr_cv = -cross_val_score(LinearRegression(), wpe_x_train, wpe_y_train, cv=10, scoring='neg_mean_squared_error')
print(f'\nThe average RMSE for Linear Regression is: {np.mean(np.sqrt(lr_cv))}')

sgd_cv = -cross_val_score(SGDRegressor(), wpe_x_train, wpe_y_train, cv=10, scoring='neg_mean_squared_error')
print(f'\nThe average RMSE for SGD is: {np.mean(np.sqrt(sgd_cv))}')

ridge_cv = -cross_val_score(Ridge(), wpe_x_train, wpe_y_train, cv=10, scoring='neg_mean_squared_error')
print(f'\nThe average RMSE for Ridge Regression is: {np.mean(np.sqrt(ridge_cv))}')

lasso_cv = -cross_val_score(Lasso(), wpe_x_train, wpe_y_train, cv=10, scoring='neg_mean_squared_error')
print(f'\nThe average RMSE for Lasso Regression is: {np.mean(np.sqrt(lasso_cv))}')

elastic_cv = -cross_val_score(ElasticNet(), wpe_x_train, wpe_y_train, cv=10, scoring='neg_mean_squared_error')
print(f'\nThe average RMSE for ElasticNet Regression is: {np.mean(np.sqrt(elastic_cv))}')

dt_cv = -cross_val_score(DecisionTreeRegressor(), wpe_x_train, wpe_y_train, cv=10, scoring='neg_mean_squared_error')
print(f'\nThe average RMSE for Decision Tree Regression is: {np.mean(np.sqrt(dt_cv))}')

rf_cv = -cross_val_score(RandomForestRegressor(), wpe_x_train, wpe_y_train, cv=10, scoring='neg_mean_squared_error')
print(f'\nThe average RMSE for Random Forest Regression is: {np.mean(np.sqrt(rf_cv))}')

svr_cv = -cross_val_score(SVR(), wpe_x_train, wpe_y_train, cv=10, scoring='neg_mean_squared_error')
print(f'\nThe average RMSE for SVR is: {np.mean(np.sqrt(svr_cv))}')

ada_cv = -cross_val_score(AdaBoostRegressor(), wpe_x_train, wpe_y_train, cv=10, scoring='neg_mean_squared_error')
print(f'\nThe average RMSE for AdaBoost Regressor is: {np.mean(np.sqrt(ada_cv))}')

gaussian_cv = -cross_val_score(GaussianProcessRegressor(), wpe_x_train, wpe_y_train, cv=10,
                               scoring='neg_mean_squared_error')
print(f'\nThe average RMSE for Gaussian Process Regressor is: {np.mean(np.sqrt(gaussian_cv))}')

In [ ]:

# plot cross validation results
def plotCvResults(model_cv):
    cv_results = pd.DataFrame(model_cv.cv_results_)
    cv_results['param_alpha'] = cv_results['param_alpha'].astype('int32')
    plt.plot(np.log1p(cv_results['param_alpha']), cv_results['mean_train_score'])
    plt.plot(np.log1p(cv_results['param_alpha']), cv_results['mean_test_score'])
    plt.xlabel('log1p(alpha)')
    plt.ylabel('Negative Mean Absolute Error')
    plt.title("Negative Mean Absolute Error and log1p(alpha)")
    plt.legend(['train score', 'test score'], loc='upper left')
    plt.show()

# display parameters
def bestParams(model, X_train):
    print("Best Alpha for Regularized Regression:", model.get_params()['alpha'])
    model_parameters = [abs(x) for x in list(model.coef_)]
    model_parameters.insert(0, model.intercept_)
    model_parameters = [round(x, 3) for x in model_parameters]
    cols = X_train.columns
    cols = cols.insert(0, "constant")
    model_coef = sorted(list(zip(cols, model_parameters)), key=operator.itemgetter(1), reverse=True)[:11]
    print("Top 10 Model parameters (excluding constant) are:")
    for p, c in model_coef:
        print(p)

def modelR2AndSpread(model, X_train, X_test, y_train, y_test):
    y_train_pred = model.predict(X_train)
    print("Train R^2:", r2_score(y_true=y_train, y_pred=y_train_pred))
    y_test_pred = model.predict(X_test)
    print("Test R^2:", r2_score(y_true=y_test, y_pred=y_test_pred))
    print('RMSE Train: ' + str(np.sqrt(mean_squared_error(y_train, y_train_pred))))
    print('RMSE Test: ' + str(np.sqrt(mean_squared_error(y_test, y_test_pred))))

    fig = plt.figure(figsize=(16, 10))
    plt.suptitle("Linear Regression Assumptions", fontsize=16)

    # plot error spread
    fig.add_subplot(2, 2, 1)
    sns.regplot(y_train, y_train_pred)
    plt.title('y_train vs y_train_pred', fontsize=14)
    plt.xlabel('y_train', fontsize=12)
    plt.ylabel('y_train_pred', fontsize=12)

    fig.add_subplot(2, 2, 2)
    sns.regplot(y_test, y_test_pred)
    plt.title('y_test vs y_test_pred', fontsize=14)
    plt.xlabel('y_test', fontsize=12)
    plt.ylabel('y_test_pred', fontsize=12)

    # plot residuals for linear regression assumption
    residuals_train = y_train - y_train_pred
    fig = plt.figure(figsize=(16, 6))
    fig.add_subplot(2, 2, 3)
    sns.distplot(residuals_train)
    plt.title('residuals between y_train & y_train_pred', fontsize=14)
    plt.xlabel('residuals', fontsize=12)

    fig.add_subplot(2, 2, 4)
    residuals_test = y_test - y_test_pred
    sns.distplot(residuals_test)
    plt.title('residuals between y_test & y_test_pred', fontsize=14)
    plt.xlabel('residuals', fontsize=12)
    plt.show()


In [ ]:
# Linear Regression 

lin_reg = LinearRegression()
lin_reg.fit(wpe_x_train, wpe_y_train)

print('MAE: ', metrics.mean_absolute_error(wpe_y_test, lin_reg.predict(wpe_x_test)))
print('MSE: ', metrics.mean_squared_error(wpe_y_test, lin_reg.predict(wpe_x_test)))
print('RMSE:', np.sqrt(metrics.mean_squared_error(wpe_y_test, lin_reg.predict(wpe_x_test))))
modelR2AndSpread(lin_reg, wpe_x_train, wpe_x_test, wpe_y_train, wpe_y_test)

In [ ]:
# Determine Ridge Regression optimal parameters

param_grid = {'alpha': (np.logspace(-8, 8, 100))}  # It will check from 1e-08 to 1e+08

ridge = Ridge(normalize=True)
ridge_model = GridSearchCV(ridge, param_grid, cv=10)
ridge_model.fit(wpe_x_train, wpe_y_train)

print(ridge_model.best_params_)
print(ridge_model.best_score_)


In [ ]:
# Ridge Regression

ridge_cv = Ridge(alpha=5.214008287999695e-05, normalize=True)
ridge_cv.fit(wpe_x_train, wpe_y_train)

print('MAE: ', metrics.mean_absolute_error(wpe_y_test, ridge_cv.predict(wpe_x_test)))
print('MSE: ', metrics.mean_squared_error(wpe_y_test, ridge_cv.predict(wpe_x_test)))
print('RMSE:', np.sqrt(metrics.mean_squared_error(wpe_y_test, ridge_cv.predict(wpe_x_test))))
modelR2AndSpread(ridge_cv, wpe_x_train, wpe_x_test, wpe_y_train, wpe_y_test)


In [ ]:
# Plot Coefficients in the Ridge Model

coef = pd.Series(ridge_cv.coef_, index=wpe_x_train.columns)
imp_coef = pd.concat([coef.sort_values()])
plt.figure(figsize=(15, 10), dpi=100)
imp_coef.plot(kind="barh")
plt.title("Coefficients in the Ridge Model")

In [ ]:
# SVR

svr = SVR()
svr_random_params = {
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
    'C': [0.001, 0.01, 0.1, 1, 10, 100],
    'gamma': [0.001, 0.01, 0.1, 1, 'scale']
}
svr_cv = GridSearchCV(svr, param_grid=svr_random_params,
                      scoring='neg_mean_absolute_error', n_jobs=-1, verbose=4)

svr_cv.fit(wpe_x_train, wpe_y_train)

print(svr_cv.best_params_)
print(svr_cv.best_score_)

In [ ]:
# Determine Decision Tree optimal parameters
# Max-depth

test_r2_score = []
train_r2_score = []
for i in range(1, 50):
    reg = DecisionTreeRegressor(max_depth=i)
    reg.fit(wpe_x_train, wpe_y_train)
    test_r2_score.append(r2_score(wpe_y_test, reg.predict(wpe_x_test)))
    train_r2_score.append(r2_score(wpe_y_train, reg.predict(wpe_x_train)))
max_depth_df = pd.DataFrame(
    {'max_depth': np.arange(1, 50), 'test_r2_score': test_r2_score, 'train_r2_score': train_r2_score})
plt.figure(figsize=(12, 6))
plt.plot(max_depth_df['max_depth'], max_depth_df['test_r2_score'], marker='o', label='test_r2_score')
plt.plot(max_depth_df['max_depth'], max_depth_df['train_r2_score'], marker='o', label='train_r2_score')
plt.legend()
plt.xlabel('max_depth')
plt.ylabel('r2_score')
plt.show()


In [ ]:
# Min_samples_split

Min_samples_split_test_r2_score = []
Min_samples_split_train_r2_score = []
for i in np.linspace(0.1, 1, 10, endpoint=True):
    reg = DecisionTreeRegressor(min_samples_split=i)
    reg.fit(wpe_x_train, wpe_y_train)
    Min_samples_split_test_r2_score.append(r2_score(wpe_y_test, reg.predict(wpe_x_test)))
    Min_samples_split_train_r2_score.append(r2_score(wpe_y_train, reg.predict(wpe_x_train)))
Min_samples_split = pd.DataFrame(
    {'Min_samples_split': np.linspace(0.1, 1, 10), 'Min_samples_split_test_r2_score': Min_samples_split_test_r2_score,
     'Min_samples_split_train_r2_score': Min_samples_split_train_r2_score})
plt.figure(figsize=(12, 6))
plt.plot(Min_samples_split['Min_samples_split'], Min_samples_split['Min_samples_split_test_r2_score'], marker='o',
         label='Min_samples_split_test_r2_score')
plt.plot(Min_samples_split['Min_samples_split'], Min_samples_split['Min_samples_split_train_r2_score'], marker='o',
         label='Min_samples_split_train_r2_score')
plt.legend()
plt.xlabel('min_samples_split')
plt.ylabel('r2_score')
plt.show()

In [ ]:
# Min_samples_leaf

min_samples_leaf_test_r2_score = []
min_samples_leaf_train_r2_score = []
for i in np.linspace(0.1, 0.5, 5, endpoint=True):
    reg = DecisionTreeRegressor(min_samples_leaf=i)
    reg.fit(wpe_x_train, wpe_y_train)
    min_samples_leaf_test_r2_score.append(r2_score(wpe_y_test, reg.predict(wpe_x_test)))
    min_samples_leaf_train_r2_score.append(r2_score(wpe_y_train, reg.predict(wpe_x_train)))
min_samples_leaf = pd.DataFrame(
    {'min_samples_leaf': np.linspace(0.1, 0.5, 5), 'min_samples_leaf_test_r2_score': min_samples_leaf_test_r2_score,
     'min_samples_leaf_train_r2_score': min_samples_leaf_train_r2_score})
plt.figure(figsize=(12, 6))
plt.plot(min_samples_leaf['min_samples_leaf'], min_samples_leaf['min_samples_leaf_test_r2_score'], marker='o',
         label='min_samples_leaf_test_r2_score')
plt.plot(min_samples_leaf['min_samples_leaf'], min_samples_leaf['min_samples_leaf_train_r2_score'], marker='o',
         label='min_samples_leaf_train_r2_score')
plt.legend()
plt.xlabel('min_samples_leaf')
plt.ylabel('r2_score')
plt.show()

In [ ]:
# Max_features

max_features_test_r2_score = []
max_features_train_r2_score = []
for i in list(range(1, wpe_df_x.shape[1])):
    reg = DecisionTreeRegressor(max_features=i)
    reg.fit(wpe_x_train, wpe_y_train)
    max_features_test_r2_score.append(r2_score(wpe_y_test, reg.predict(wpe_x_test)))
    max_features_train_r2_score.append(r2_score(wpe_y_train, reg.predict(wpe_x_train)))
max_features = pd.DataFrame(
    {'max_features': list(range(1, wpe_df_x.shape[1])), 'max_features_test_r2_score': max_features_test_r2_score,
     'max_features_train_r2_score': max_features_train_r2_score})

plt.figure(figsize=(12, 6))
plt.plot(np.arange(1, 75), max_features['max_features_test_r2_score'], marker='o', label='max_features_test_r2_score')
plt.plot(np.arange(1, 75), max_features['max_features_train_r2_score'], marker='o', label='max_features_train_r2_score')
plt.legend()
plt.xlabel('max_features')
plt.ylabel('r2_score')
plt.show()

In [ ]:
# Max_leaf_nodes

max_leaf_nodes_test_r2_score = []
max_leaf_nodes_train_r2_score = []
for i in np.arange(1, 110, 10):
    reg = DecisionTreeRegressor(min_samples_leaf=i)
    reg.fit(wpe_x_train, wpe_y_train)
    max_leaf_nodes_test_r2_score.append(r2_score(wpe_y_test, reg.predict(wpe_x_test)))
    max_leaf_nodes_train_r2_score.append(r2_score(wpe_y_train, reg.predict(wpe_x_train)))
max_leaf_nodes = pd.DataFrame(
    {'min_samples_leaf': np.arange(1, 110, 10), 'max_leaf_nodes_test_r2_score': max_leaf_nodes_test_r2_score,
     'max_leaf_nodes_train_r2_score': max_leaf_nodes_train_r2_score})
plt.figure(figsize=(12, 6))
plt.plot(np.arange(1, 110, 10), max_leaf_nodes['max_leaf_nodes_test_r2_score'], marker='o',
         label='max_leaf_nodes_test_r2_score')
plt.plot(np.arange(1, 110, 10), max_leaf_nodes['max_leaf_nodes_train_r2_score'], marker='o',
         label='max_leaf_nodes_train_r2_score')
plt.legend()
plt.xlabel('max_leaf_nodes')
plt.ylabel('r2_score')
plt.show()

In [ ]:
# Run Decision Tree Regressor with basic Grid Search parameters
# Ignores previous cells that calculated Max-depth, Min_samples_split, Min_samples_leaf, Max_features, Max_leaf_nodes

dt = DecisionTreeRegressor()

grid = {'max_depth': np.arange(10, 30),
        "min_samples_leaf": np.arange(1, 100, 10),
        "min_samples_split": np.linspace(0.001, 1, 10)}

dt_grid = GridSearchCV(estimator=dt, param_grid=grid, verbose=4)
dt_grid.fit(wpe_x_train, wpe_y_train)

print('Train R^2 Score : %.3f' % dt_grid.best_estimator_.score(wpe_x_train, wpe_y_train))
print('Test R^2 Score : %.3f' % dt_grid.best_estimator_.score(wpe_x_test, wpe_y_test))
print('Best R^2 Score Through Grid Search : %.3f' % dt_grid.best_score_)
print('Best Parameters : ', dt_grid.best_params_)

print('MAE: ', metrics.mean_absolute_error(wpe_y_test, dt_grid.predict(wpe_x_test)))
print('MSE: ', metrics.mean_squared_error(wpe_y_test, dt_grid.predict(wpe_x_test)))
print('RMSE:', np.sqrt(metrics.mean_squared_error(wpe_y_test, dt_grid.predict(wpe_x_test))))
modelR2AndSpread(dt_grid, wpe_x_train, wpe_x_test, wpe_y_train, wpe_y_test)

In [ ]:
# Plot feature importances from DT regressor

FI_df = pd.DataFrame({'Features': wpe_x_train.columns, 'Importance': dt_grid.best_estimator_.feature_importances_})
FI_df = FI_df.sort_values(by='Importance', ascending=False).head(100)

plt.figure(figsize=(15, 10), dpi=100)
sns.barplot(x=FI_df[:30].Importance, y=FI_df[:30].Features, orient='h').set_title('Feature Importance')


In [ ]:
# Random Forest Regressor

rf = RandomForestRegressor()
rf.fit(wpe_x_train, wpe_y_train)

print('MAE: ', metrics.mean_absolute_error(wpe_y_test, rf.predict(wpe_x_test)))
print('MSE: ', metrics.mean_squared_error(wpe_y_test, rf.predict(wpe_x_test)))
print('RMSE:', np.sqrt(metrics.mean_squared_error(wpe_y_test, rf.predict(wpe_x_test))))
modelR2AndSpread(rf, wpe_x_train, wpe_x_test, wpe_y_train, wpe_y_test)


In [ ]:
# Plot feature importances from RF regressor

FI_df = pd.DataFrame({'Features': wpe_x_train.columns, 'Importance': rf.feature_importances_})
FI_df = FI_df.sort_values(by='Importance', ascending=False).head(100)

plt.figure(figsize=(15, 10), dpi=100)
sns.barplot(x=FI_df[:30].Importance, y=FI_df[:30].Features, orient='h').set_title('Feature Importance')

In [ ]:
# Shap Values
# Final step

explainer = shap.TreeExplainer(rf)
print(explainer)

shap_values = explainer.shap_values(wpe_x_test)
print(shap_values)

shap.summary_plot(shap_values, wpe_x_test)

In [ ]:
# (Still working on!)
# Purpose:
# Create faster approach for hyperparamter turning using RandomizedSearchCV
# All models are then ran with optimal hyperparameters
# Data frame is constructed with evaluation metrics for each model

n_iter_num = 20
cv_num = 3

def train_LinearRegression(X_train, y_train):
    print('Training Linear Regression ...')
    lr = LinearRegression(copy_X=True, positive=False, n_jobs=-1)
    lr_random_params = {
        'fit_intercept': [True, False],
        'normalize': [True, False]
    }
    lr_cv = RandomizedSearchCV(lr, param_distributions=lr_random_params, n_iter=n_iter_num, cv=cv_num,
                               scoring='neg_mean_absolute_error', n_jobs=-1, random_state=42, verbose=4)
    lr_cv.fit(X_train, y_train)
    return lr_cv


# higher value for alpha implies that there were some useless features in the model and therefore by increasing alpha the affect of these features has been decreased, decreasing the variance of the model (the model will not overfit as severly)

def train_Lasso(X_train, y_train):
    print('Training Lasso Regression ...')
    lasso = Lasso()
    lasso_random_params = {
        'alpha': [0.0001, 0.001, 0.005, 0.01, 0.03, 0.05, 0.1, 0.5, 1.0, 5.0, 10]
    }
    lasso_cv = RandomizedSearchCV(lasso, param_distributions=lasso_random_params, n_iter=n_iter_num, cv=cv_num,
                                  scoring='neg_mean_absolute_error', n_jobs=-1, random_state=42, verbose=4)
    lasso_cv.fit(X_train, y_train)
    return lasso_cv


def train_Ridge(X_train, y_train):
    print('Training Ridge Regression ...')
    ridge = Ridge()
    ridge_random_params = {
        'alpha': [0.0001, 0.001, 0.005, 0.01, 0.03, 0.05, 0.1, 0.5, 1.0, 5.0, 10]
    }
    ridge_cv = RandomizedSearchCV(ridge, param_distributions=ridge_random_params, n_iter=n_iter_num, cv=cv_num,
                                  scoring='neg_mean_absolute_error', n_jobs=-1, random_state=42, verbose=4)
    ridge_cv.fit(X_train, y_train)
    return ridge_cv


def train_ElasticNet(X_train, y_train):
    print('Training Elastic Net ...')
    elastic_net = ElasticNet()
    elastic_net_random_params = {
        'model__alpha': list(np.arange(0, 20, 0.1)),
        'model__l1_ratio': list(np.arange(0, 1, 0.001))
    }
    elastic_net_cv = RandomizedSearchCV(elastic_net, param_distributions=elastic_net_random_params, n_iter=n_iter_num,
                                        cv=cv_num,
                                        scoring='neg_mean_absolute_error', n_jobs=-1, random_state=42, verbose=4)
    elastic_net_cv.fit(X_train, y_train)
    return elastic_net_cv


def train_SVR(X_train, y_train):
    print('Training SVR ...')
    svr = SVR()
    svr_random_params = {
        'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
        'C': scipy.stats.reciprocal(0.001, 100.),
        'epsilon': scipy.stats.uniform(0.1, 1.),
        'gamma': scipy.stats.reciprocal(0.001, 1.),
    }
    svr_cv = RandomizedSearchCV(svr, param_distributions=svr_random_params, n_iter=n_iter_num, cv=cv_num,
                                scoring='neg_mean_absolute_error', n_jobs=-1, random_state=42, verbose=4)
    svr_cv.fit(X_train, y_train)
    return svr_cv


def train_DecisionTree(X_train, y_train):
    print('Training DecisionTree ...')
    dt = DecisionTreeRegressor(random_state=0)
    dt_random_params = {
        'max_depth': scipy.stats.randint(10, 100)
    }
    dt_cv = sklearn.model_selection.RandomizedSearchCV(dt, param_distributions=dt_random_params, n_iter=n_iter_num,
                                                       cv=cv_num,
                                                       scoring='neg_mean_absolute_error', n_jobs=-1, random_state=42,
                                                       verbose=4)
    dt_cv.fit(X_train, y_train)
    return dt_cv


def train_RandomForest(X_train, y_train):
    print('Training RandomForest ...')
    rf = RandomForestRegressor()

    n_estimators = [int(x) for x in np.linspace(start=100, stop=1200, num=12)]
    features = ['auto', 'sqrt']
    depth = [int(x) for x in np.linspace(5, 30, num=6)]
    samples_split = [2, 5, 10, 15, 100]
    sample_leaf = [1, 2, 5, 10]

    rf_random_params = {
        'n_estimators': n_estimators,
        'max_features': features,
        'max_depth': depth,
        'min_samples_split': samples_split,
        'min_samples_leaf': sample_leaf
    }
    rf_cv = sklearn.model_selection.RandomizedSearchCV(rf, param_distributions=rf_random_params, n_iter=n_iter_num,
                                                       cv=cv_num,
                                                       scoring='neg_mean_absolute_error', n_jobs=-1, random_state=42,
                                                       verbose=4)
    rf_cv.fit(X_train, y_train)
    return rf_cv


def train_AdaBoost(X_train, y_train):
    print('Training AdaBoost ...')
    ada_boost = AdaBoostRegressor(random_state=0)
    ada_boost_random_params = {
        'loss': ['linear', 'square', 'exponential'],
        'learning_rate': scipy.stats.uniform(0.75, 1.25),
        'n_estimators': scipy.stats.randint(40, 100)
    }
    ada_boost_cv = sklearn.model_selection.RandomizedSearchCV(ada_boost, param_distributions=ada_boost_random_params,
                                                              n_iter=n_iter_num, cv=cv_num,
                                                              scoring='neg_mean_absolute_error', n_jobs=-1,
                                                              random_state=42, verbose=4)
    ada_boost_cv.fit(X_train, y_train)
    return ada_boost_cv


def train_GaussianProcess(X_train, y_train):
    print('Training GaussianProcess ...')
    alpha = 1e-9
    gaussian = GaussianProcessRegressor(normalize_y=True, random_state=0, optimizer=None, alpha=alpha)
    gaussian_random_params = {
        'kernel': [DotProduct(), WhiteKernel(), RBF(), Matern(), RationalQuadratic()],
        'n_restarts_optimizer': scipy.stats.randint(0, 10),
        #         'alpha' : scipy.stats.uniform(1e-9, 1e-8)
    }
    gaussian_cv = sklearn.model_selection.RandomizedSearchCV(gaussian, param_distributions=gaussian_random_params,
                                                             n_iter=n_iter_num, cv=cv_num,
                                                             scoring='neg_mean_absolute_error', n_jobs=-1,
                                                             random_state=42, verbose=4)
    gaussian_cv.fit(X_train, y_train)
    return gaussian_cv


def run_all_regrs(X_train, y_train):
    all_regrs = []
    regr_names = []

    svm = train_SVR(X_train, y_train)
    all_regrs.append(svm.best_estimator_)
    regr_names.append('SVR')

    decision_tree = train_DecisionTree(X_train, y_train)
    all_regrs.append(decision_tree.best_estimator_)
    regr_names.append('Decision Tree')

    random_forest = train_RandomForest(X_train, y_train)
    all_regrs.append(random_forest.best_estimator_)
    regr_names.append('Random Forest')

    ada_boost = train_AdaBoost(X_train, y_train)
    all_regrs.append(ada_boost.best_estimator_)
    regr_names.append('AdaBoost')

    gaussian_process = train_GaussianProcess(X_train, y_train)
    all_regrs.append(gaussian_process.best_estimator_)
    regr_names.append('Gaussian Process')

    linear_regression = train_LinearRegression(X_train, y_train)
    all_regrs.append(linear_regression.best_estimator_)
    regr_names.append('Linear Regression')

    print(all_regrs)
    print(regr_names)

    return svm, decision_tree, random_forest, ada_boost, gaussian_process, linear_regression


def model_evaluation(models, X_train, y_train, X_test, y_test, X_df, y_df, cv_num):
    cross_val = []
    r2_train = []
    r2_test = []
    mae_score = []
    rmse_score = []
    best_estimator = []
    regr_names = []

    for model in models:
        cv = cross_val_score(estimator=model, X=X_df, y=y_df, cv=cv_num)
        y_pred_train = model.predict(X_train)
        r2_score_train = r2_score(y_train, y_pred_train)
        y_pred_test = model.predict(X_test)
        r2_score_test = r2_score(y_test, y_pred_test)
        rmse = (np.sqrt(mean_squared_error(y_test, y_pred_test)))
        mae = mean_absolute_error(y_test, y_pred_test)
        cross_val.append(cv.mean())
        r2_train.append(r2_score_train)
        r2_test.append(r2_score_test)
        mae_score.append(mae)
        rmse_score.append(rmse)
        best_estimator.append(model.best_estimator_)
        regr_names.append(model)

    metrics_df = pd.DataFrame(
        list(zip(cross_val, r2_train, r2_test, mae_score, rmse_score, best_estimator, regr_names)),
        columns=['Cross Validation Score', 'R^2 Train', 'R^2 Test', 'MAE', 'RMSE', 'Best estimator',
                 'Regression Model'])
    return metrics_df